### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

660510751 กิตติพล ทาอิ่นคำ

In [1]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


In [2]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [3]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define transformation
tfidf = TfidfVectorizer()

# fit+transform training set
Xtrain_tfidf = tfidf.fit_transform(Xtrain)

# See output
Xtrain_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 470232 stored elements and shape (3000, 61994)>

In [5]:
#tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [6]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier(n_neighbors=5)

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)

KNeighborsClassifier()

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [7]:
from sklearn.metrics import classification_report

# Transform the test set
Xtest_tfidf = tfidf.transform(Xtest)

# Make predictions on the test set
ypred = nb.predict(Xtest_tfidf)

# Report classification scores





                          precision    recall  f1-score   support

             alt.atheism       0.18      0.48      0.26        21
           comp.graphics       0.38      0.48      0.43        21
 comp.os.ms-windows.misc       0.56      0.58      0.57        26
comp.sys.ibm.pc.hardware       0.52      0.50      0.51        34
   comp.sys.mac.hardware       0.67      0.53      0.59        34
          comp.windows.x       0.88      0.54      0.67        26
            misc.forsale       0.50      0.41      0.45        22
               rec.autos       0.58      0.54      0.56        28
         rec.motorcycles       0.74      0.70      0.72        33
      rec.sport.baseball       0.82      0.72      0.77        25
        rec.sport.hockey       0.75      0.56      0.64        27
               sci.crypt       0.71      0.85      0.77        20
         sci.electronics       0.31      0.21      0.25        24
                 sci.med       0.79      0.48      0.59        23
         

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pipeline = Pipeline([('tfidf', TfidfVectorizer()),('nb', KNeighborsClassifier(n_neighbors=5))])

# Fit the pipeline to the training set
pipeline.fit(Xtrain, ytrain)

# Make predictions on the test set
ypred = pipeline.predict(Xtest)

# report classification scores
print(classification_report(ytest, ypred, target_names=train.target_names))



                          precision    recall  f1-score   support

             alt.atheism       0.18      0.48      0.26        21
           comp.graphics       0.38      0.48      0.43        21
 comp.os.ms-windows.misc       0.56      0.58      0.57        26
comp.sys.ibm.pc.hardware       0.52      0.50      0.51        34
   comp.sys.mac.hardware       0.67      0.53      0.59        34
          comp.windows.x       0.88      0.54      0.67        26
            misc.forsale       0.50      0.41      0.45        22
               rec.autos       0.58      0.54      0.56        28
         rec.motorcycles       0.74      0.70      0.72        33
      rec.sport.baseball       0.82      0.72      0.77        25
        rec.sport.hockey       0.75      0.56      0.64        27
               sci.crypt       0.71      0.85      0.77        20
         sci.electronics       0.31      0.21      0.25        24
                 sci.med       0.79      0.48      0.59        23
         

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [18]:
from sklearn.model_selection import GridSearchCV

params = {'tfidf__ngram_range': [(1, 1), (1, 2)],
    'nb__n_neighbors': [1, 3, 5, 7],
    'nb__weights': ['uniform', 'distance']}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv=3)

# Fit and cross-validate the model on 3-fold data
gridcv.fit(Xtrain, ytrain)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('nb', KNeighborsClassifier())]),
             param_grid={'nb__n_neighbors': [1, 3, 5, 7],
                         'nb__weights': ['uniform', 'distance'],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')

In [20]:
gridcv.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('nb', KNeighborsClassifier(n_neighbors=1))])

In [19]:
# Make predictions on the test set
ypred = gridcv.predict(Xtest)

# Report classification scores
print(classification_report(ytest, ypred, target_names=train.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.29      0.52      0.37        21
           comp.graphics       0.38      0.43      0.40        21
 comp.os.ms-windows.misc       0.54      0.54      0.54        26
comp.sys.ibm.pc.hardware       0.46      0.38      0.42        34
   comp.sys.mac.hardware       0.81      0.50      0.62        34
          comp.windows.x       0.68      0.50      0.58        26
            misc.forsale       0.41      0.41      0.41        22
               rec.autos       0.67      0.57      0.62        28
         rec.motorcycles       0.82      0.70      0.75        33
      rec.sport.baseball       0.69      0.72      0.71        25
        rec.sport.hockey       0.68      0.56      0.61        27
               sci.crypt       0.70      0.80      0.74        20
         sci.electronics       0.43      0.42      0.43        24
                 sci.med       0.65      0.57      0.60        23
         

#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [22]:
from sklearn.metrics import f1_score

params = {
    'nb__n_neighbors': [1, 3, 5, 7],
    'nb__metric': ['euclidean', 'manhattan', 'cosine']
}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params, scoring='f1_macro', cv=5)

# Fit and cross-validate the model on 5-fold data
gridcv.fit(Xtrain, ytrain)

# Make predictions on the test set with the best estimator
ypred = gridcv.best_estimator_.predict(Xtest)

# Compute and print the f1_macro score on the test set
f1_macro_score = f1_score(ytest, ypred, average='macro')
print("F1-macro score on test set:", f1_macro_score)
print("Best n_neighbors:", gridcv.best_params_['nb__n_neighbors'])
print("Best metric:", gridcv.best_params_['nb__metric'])

F1-macro score on test set: 0.5497051947955395
Best n_neighbors: 1
Best metric: euclidean
